In [1]:
from database_utils import DatabaseConnector
from data_extraction import DataExtractor
from dateutil.parser import parse
import pandas as pd

In [2]:
#intializing engine
connector = DatabaseConnector()
engine1, engine2 = connector.init_db_engine()

In [3]:
#extracting the card details from the cloud
extractor = DataExtractor()
pdf_path = 'https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf'
extracted_card_df = extractor.retrieve_pdf_data(pdf_path)
extracted_card_df.info()


c:\Users\mohdi\Miniconda3\envs\mrdc\Lib\site-packages\tabula\io.py:1045: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15309 entries, 0 to 15308
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   card_number             15309 non-null  object
 1   expiry_date             15309 non-null  object
 2   card_provider           15309 non-null  object
 3   date_payment_confirmed  15309 non-null  object
dtypes: object(4)
memory usage: 478.5+ KB


In [4]:
print(extracted_card_df.date_payment_confirmed.head())
print(f"Value count is: \n{extracted_card_df.date_payment_confirmed.value_counts()}")

0    2015-11-25
1    2001-06-18
2    2000-12-26
3    2011-02-12
4    1997-03-13
Name: date_payment_confirmed, dtype: object
Value count is: 
date_payment_confirmed
NULL          11
2009-12-20     8
1998-02-25     8
2017-03-31     7
2000-09-26     7
              ..
2016-09-18     1
2007-01-11     1
2021-07-09     1
2015-05-24     1
2009-02-04     1
Name: count, Length: 8270, dtype: int64


In [5]:
#we mask out these NULLS for the entire dataframe
Nulls = ['NULL']
mask = extracted_card_df.date_payment_confirmed.isin(Nulls)
subset_df = extracted_card_df[mask]
subset_df

,card_number,expiry_date,card_provider,date_payment_confirmed
377,NULL,NULL,NULL,NULL
847,NULL,NULL,NULL,NULL
884,NULL,NULL,NULL,NULL
2418,NULL,NULL,NULL,NULL
2489,NULL,NULL,NULL,NULL
2830,NULL,NULL,NULL,NULL
4196,NULL,NULL,NULL,NULL
5686,NULL,NULL,NULL,NULL
6024,NULL,NULL,NULL,NULL
12876,NULL,NULL,NULL,NULL


In [6]:
#from above we can see that there are 11NULL values present in this column which looking upon closely are entirely NULL for each column in the entire dataframe
extracted_card_df.dropna(subset=['date_payment_confirmed'], inplace=True)
#This doesnt drop nulls because they are not recognised by pandas because they are string
print(extracted_card_df.date_payment_confirmed.iloc[377])
#we check if NULLS are dropped. But they arent

NULL


In [7]:
#we have to manually handle these NULL values in the dataframe because they might be string or in a format pandas cannot recognise.
extracted_card_df.replace('NULL', float("NaN"), inplace= True)
extracted_card_df.replace('', float("NaN"), inplace= True)
extracted_card_df.replace(' ', float("NaN"), inplace= True)
print(extracted_card_df.date_payment_confirmed.iloc[377])

nan


In [8]:
extracted_card_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15309 entries, 0 to 15308
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   card_number             15298 non-null  object
 1   expiry_date             15298 non-null  object
 2   card_provider           15298 non-null  object
 3   date_payment_confirmed  15298 non-null  object
dtypes: object(4)
memory usage: 478.5+ KB


In [9]:
#dropping rows with all NaN values
extracted_card_df.dropna(axis = 0, how='all', inplace= True)

In [10]:
extracted_card_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15298 entries, 0 to 15308
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   card_number             15298 non-null  object
 1   expiry_date             15298 non-null  object
 2   card_provider           15298 non-null  object
 3   date_payment_confirmed  15298 non-null  object
dtypes: object(4)
memory usage: 597.6+ KB


In [11]:
extracted_card_df.card_provider.value_counts()

card_provider
VISA 16 digit                  2426
JCB 16 digit                   2403
VISA 13 digit                  1382
JCB 15 digit                   1363
VISA 19 digit                  1349
Diners Club / Carte Blanche    1312
American Express               1297
Maestro                        1281
Discover                       1260
Mastercard                     1211
OGJTXI6X1H                        1
BU9U947ZGV                        1
UA07L7EILH                        1
XGZBYBYGUW                        1
DLWF2HANZF                        1
1M38DYQTZV                        1
JRPRLPIBZ2                        1
DE488ORDXY                        1
5CJH7ABGDR                        1
JCQMU8FN85                        1
TS8A81WFXV                        1
WJVMUO4QX6                        1
NB71VBAHJE                        1
5MFWFBZRM9                        1
Name: count, dtype: int64

In [12]:
#Removing the gibberish values from the dataframe and saving it in a filtered dataframe namely df.
gibberish_values = ['OGJTXI6X1H', 'BU9U947ZGV', 'UA07L7EILH', 'XGZBYBYGUW', 'DLWF2HANZF', '1M38DYQTZV', 'JRPRLPIBZ2',  'DE488ORDXY', '5CJH7ABGDR', 'JCQMU8FN85', 'TS8A81WFXV', 'WJVMUO4QX6', 'NB71VBAHJE', '5MFWFBZRM9']
df = extracted_card_df[~extracted_card_df.card_provider.isin(gibberish_values)]
df

,card_number,expiry_date,card_provider,date_payment_confirmed
0,30060773296197,09/26,Diners Club / Carte Blanche,2015-11-25
1,349624180933183,10/23,American Express,2001-06-18
2,3529023891650490,06/23,JCB 16 digit,2000-12-26
3,213142929492281,09/27,JCB 15 digit,2011-02-12
4,502067329974,10/25,Maestro,1997-03-13
...,...,...,...,...
15304,180036921556789,12/28,JCB 15 digit,1997-06-06
15305,180018030448512,11/24,JCB 15 digit,2004-06-16
15306,3569953313547220,04/24,JCB 16 digit,2020-02-05
15307,4444521712606810,06/27,VISA 16 digit,2008-06-16


In [13]:
date_format = '%m/%y'
df.expiry_date = pd.to_datetime(df.expiry_date, format = date_format, errors= 'coerce')
df.expiry_date

C:\Users\mohdi\AppData\Local\Temp\ipykernel_16848\4045507631.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.expiry_date = pd.to_datetime(df.expiry_date, format = date_format, errors= 'coerce')


0       2026-09-01
1       2023-10-01
2       2023-06-01
3       2027-09-01
4       2025-10-01
           ...    
15304   2028-12-01
15305   2024-11-01
15306   2024-04-01
15307   2027-06-01
15308   2030-02-01
Name: expiry_date, Length: 15284, dtype: datetime64[ns]

In [27]:
#applying parse because there are inconsistent date strings in this column, due to which format fails.
df.date_payment_confirmed = df.date_payment_confirmed.apply(parse)

Timestamp('2021-12-17 00:00:00')

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15285 entries, 0 to 1443
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   card_number             15284 non-null  object        
 1   expiry_date             15284 non-null  datetime64[ns]
 2   card_provider           15284 non-null  object        
 3   date_payment_confirmed  15285 non-null  datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 1.1+ MB
